<a href="https://colab.research.google.com/github/MTia05/Visualizzazione-Scientifica/blob/main/codici/confrontoUominiDonne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Colori e font
COLOR_MALE = "#5DADE2"    # Azzurro (Sinistra)
COLOR_FEMALE = "#F06292"  # Rosa (Destra)
COLOR_TEXT = "#455a64"
COLOR_GRID = "#f0f0f0"
FONT_FAMILY = "Arial, sans-serif"

def crea_grafico_piramide():
    try:
        # Caricamento dati da entrambe le tabelle
        # Trova la cartella dove si trova il notebook
        base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
        # Unisce i pezzi in modo corretto
        file_path_femmine = os.path.join(base_path, "dataSet", "datasetFemmine.csv")

        file_path_maschi = os.path.join(base_path, "dataSet", "datasetMaschi.csv")

        df_femmine = pd.read_csv(file_path_femmine, sep=',', encoding='latin-1', quotechar="'")
        df_maschi = pd.read_csv(file_path_maschi, sep=',', encoding='latin-1', quotechar="'")

        # Pulizia nomi
        for df in [df_femmine, df_maschi]:
            df.columns = df.columns.str.strip().str.replace('"', '').str.replace("'", "")

        col_anno = 'TIME_PERIOD'
        col_valore = 'Osservazione'

        # Verifica presenza colonne
        if col_anno not in df_femmine.columns or col_valore not in df_femmine.columns:
            cols = [c for c in df_femmine.columns if 'TIME' in c or 'Anno' in c]
            if cols: col_anno = cols[0]
            vals = [c for c in df_femmine.columns if 'Osserv' in c or 'Value' in c]
            if vals: col_valore = vals[0]

        # Ordinamento per anno
        femmine = df_femmine[[col_anno, col_valore]].sort_values(col_anno)
        maschi = df_maschi[[col_anno, col_valore]].sort_values(col_anno)

        # Merge per allineare gli anni
        df_final = pd.merge(maschi, femmine, on=col_anno, suffixes=('_M', '_F'))

        years = df_final[col_anno].tolist()
        val_m = df_final[f'{col_valore}_M'].tolist()
        val_f = df_final[f'{col_valore}_F'].tolist()

        # Calcolo limite massimo
        max_val = max(max(val_m), max(val_f))
        # Aumento il limite per avere più spazio
        limit = (max_val // 1000 + 1) * 1000 * 1.3

        # Tacche asse x ogni milione
        step_in_thousands = 1000
        tick_vals = []
        tick_text = []

        # Genera le tacche da -limit a +limit
        current_limit = int((max_val // 1000 + 1) * 1000)
        for i in range(-current_limit, current_limit + 1, step_in_thousands):
            tick_vals.append(i)
            # Il testo della tacca deve essere il valore assoluto in milioni
            val_abs_millions = abs(i) / 1000
            if val_abs_millions == 0:
                tick_text.append("0")
            else:
                tick_text.append(f"{val_abs_millions:.0f}M")

        # Creazione grafico
        fig = go.Figure()

        # Barre Maschi a sinistra - Valori negativi
        fig.add_trace(go.Bar(
            y=years,
            x=[-v for v in val_m],
            name='Uomini',
            orientation='h',
            marker=dict(color=COLOR_MALE),
            hovertemplate='Anno: %{y}<br>Uomini: %{customdata:.2f} milioni<extra></extra>',
            customdata=[v / 1000 for v in val_m], # Conversione in M
            text=[f'{v:.2f}M' for v in (val / 1000 for val in val_m)],
            textposition='outside',
            textfont=dict(color=COLOR_MALE) # Colorazione azzurro
        ))

        # Barre Femmine a destra - Valori positivi
        fig.add_trace(go.Bar(
            y=years,
            x=val_f,
            name='Donne',
            orientation='h',
            marker=dict(color=COLOR_FEMALE),
            hovertemplate='Anno: %{y}<br>Donne: %{customdata:.2f} milioni<extra></extra>',
            customdata=[v / 1000 for v in val_f], # Conversione
            text=[f'{v:.2f}M' for v in (val / 1000 for val in val_f)],
            textposition='outside',
            textfont=dict(color=COLOR_FEMALE) # Colorazione rosa
        ))

        # Layout e stile
        fig.update_layout(
            template="plotly_white",
            barmode='relative',
            bargap=0.3,
            width=1000,
            height=850,
            font=dict(family=FONT_FAMILY, color=COLOR_TEXT),
            margin=dict(l=100, r=100, t=160, b=80),

            xaxis=dict(
                title="Fumatori (milioni)",
                gridcolor=COLOR_GRID,
                zerolinecolor=COLOR_TEXT,
                zerolinewidth=2,
                range=[-limit, limit],
                tickvals=tick_vals,
                ticktext=tick_text,
                fixedrange=True
            ),

            yaxis=dict(
                title="Anno di rilevazione",
                tickmode='linear',
                dtick=1,
                gridcolor=COLOR_GRID,
            ),

            legend=dict(
                orientation="h",
                yanchor="bottom", y=1.02,
                xanchor="center", x=0.5,
                bgcolor="rgba(255,255,255,0.8)"
            )
        )

        # Titolo e Sottotitolo
        fig.add_annotation(
            x=0.5, y=1.22,
            text="<b style='font-size:32px; color:#2E4053;'>Confronto tra Fumatori Uomini e Donne</b>",
            showarrow=False, xref="paper", yref="paper"
        )

        fig.add_annotation(
            x=0.5, y=1.12,
            text="<span style='font-size:16px; color:#7F8C8D;'>Dati dal 2001 al 2024 • Valori espressi in milioni</span>",
            showarrow=False, xref="paper", yref="paper"
        )

        fig.show()

    except Exception as e:
        print(f"Errore durante l'elaborazione dei file: {e}")

if __name__ == "__main__":
    crea_grafico_piramide()